In [41]:
import geopandas as gpd 
import pandas as pd 
import fiona

## base parcels

In [42]:
p10_dataset_path = r'C:\Users\ywang\Documents\ArcGIS\Projects\PBA2050 Plus Blueprint Data QAQC - 20231206\PBA2050 Plus Blueprint Data QAQC\PBA2050 Plus Blueprint Data QAQC.gdb'
p10_layers = fiona.listlayers(p10_dataset_path)
print(p10_layers)

['p10_parcels_raw', 'p10_parcels_topo_fix', 'p10_urbansim_plan_geographies']


In [43]:
p10_topo_fix = gpd.read_file(p10_dataset_path, driver='fileGDB', layer='p10_parcels_topo_fix')

In [44]:
print(p10_topo_fix.shape[0])
print(p10_topo_fix['parcel_id'].nunique())

1956207
1956207


In [45]:
print(p10_topo_fix.crs)

epsg:26910


In [46]:
# get parcel centroids for spatial join

def get_centroid(x, ensure_within=True):
    """
    Given a shapely Polygon, returns the centroid.
    If ensure_within is True, and centroid is not
    within the polygon, returns the representative_point
    """
    try:
        centroid = x.centroid
        if ensure_within and not x.intersects(centroid):
            centroid = x.representative_point()
    except: pass
    return centroid


In [47]:
# get parcel centroids
p10_gdf = p10_topo_fix.copy()
p10_gdf['centroid'] = p10_gdf['geometry'].map(get_centroid)
p10_gdf.loc[p10_gdf['centroid'].isnull()].shape[0]

# create a gdf of parcel_id and centrod geometry
p10_centroid_gdf = gpd.GeoDataFrame(p10_gdf[['parcel_id']], geometry=p10_gdf['centroid'], crs='EPSG:26910')
p10_centroid_gdf.head()

,parcel_id,geometry
0,1906302.0,POINT (515952.701 4243055.845)
1,1740249.0,POINT (605099.029 4222734.303)
2,1630642.0,POINT (617528.748 4133218.640)
3,1725445.0,POINT (630054.383 4108236.029)
4,837988.0,POINT (536304.276 4201510.130)


## get Growth Geographies layers

In [48]:
# dataset that contains TRAs, GGs, PPAs, PDA source layers

dataset_path = r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\pba50plus_GrowthGeographies_p10tagging.gdb'
layers = fiona.listlayers(dataset_path)
print(layers)

['PBA50Plus_TRAs_ExportFeature', 'p10_parcels_tagging_20231205', 'p10_parcels_tag_20231205', 'PBA50Plus_GG_ExportFeatures', 'PBA50Plus_PDA_ExportFeatures', 'PPA_ExportFeatures', 'PBA50Plus_PDA_ExportFeatures_correct']


In [49]:
## TRA layer
# tra_layer = gpd.read_file(dataset_path, driver='fileGDB', layer='PBA50Plus_TRAs_ExportFeature')
tra_layer = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\PBA50_Plus_TRA_v2_121123.shp')
tra_layer.rename(columns={'PBA50_Geo': 'tra_50plus'}, inplace=True)

print(tra_layer.crs)
tra_layer_proj = tra_layer.to_crs('EPSG:26910')
print(tra_layer_proj.crs)

epsg:3857
EPSG:26910


In [50]:
# tag parcels by TRA
tra_tag = gpd.sjoin(p10_centroid_gdf, tra_layer_proj, how="left", predicate="within")
display(tra_tag.head())
tra_tag.loc[tra_tag['tra_50plus'].notnull()]

,parcel_id,geometry,index_right,tra_50plus,Shape_Leng,Shape_Area
0,1906302.0,POINT (515952.701 4243055.845),NaN,NaN,NaN,NaN
1,1740249.0,POINT (605099.029 4222734.303),NaN,NaN,NaN,NaN
2,1630642.0,POINT (617528.748 4133218.640),NaN,NaN,NaN,NaN
3,1725445.0,POINT (630054.383 4108236.029),NaN,NaN,NaN,NaN
4,837988.0,POINT (536304.276 4201510.130),NaN,NaN,NaN,NaN


,parcel_id,geometry,index_right,tra_50plus,Shape_Leng,Shape_Area
6,122690.0,POINT (560909.952 4190761.975),44.0,TRA3,209204.584759,1.224817e+08
81,1296016.0,POINT (588416.918 4140330.159),1.0,TRA2,609530.474388,3.931538e+08
173,1722821.0,POINT (625865.801 4100965.030),2.0,TRA3,23375.756857,1.052393e+07
343,1578229.0,POINT (602669.961 4129360.261),12.0,TRA3,145381.753424,7.157681e+07
362,152680.0,POINT (563454.268 4181752.229),44.0,TRA3,209204.584759,1.224817e+08
...,...,...,...,...,...,...
1956190,1674412.0,POINT (605943.303 4122230.645),1.0,TRA2,609530.474388,3.931538e+08
1956192,2050178.0,POINT (593096.607 4130816.819),14.0,TRA3,187839.030840,9.226378e+07
1956199,141351.0,POINT (564963.039 4180832.469),44.0,TRA3,209204.584759,1.224817e+08
1956204,269036.0,POINT (579926.210 4168835.977),25.0,TRA3,6392.269837,2.712776e+06


In [51]:
tra_tag.loc[tra_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,tra_50plus,Shape_Leng,Shape_Area


In [52]:
## GG layer
# gg_layer = gpd.read_file(dataset_path, driver='fileGDB', layer='PBA50Plus_GG_ExportFeatures')
gg_layer = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\PBA50Plus_Growth_Geographies_120823.shp')
# gg_layer.rename(columns={'PBA50_Geo': 'gg_50plus'}, inplace=True)
gg_layer['gg_50plus'] = 'GG'

print(gg_layer.crs)
gg_layer_proj = gg_layer.to_crs('EPSG:26910')
print(gg_layer_proj.crs)

epsg:4326
EPSG:26910


In [53]:
gg_layer

,Shape_Leng,Shape_Area,geometry,gg_50plus
0,31.073629,0.104814,"MULTIPOLYGON Z (((-121.58276 37.03564 0.00000,...",GG


In [54]:
# tag parcels by GG
gg_tag = gpd.sjoin(p10_centroid_gdf, gg_layer_proj, how="left", predicate="within")
display(gg_tag.head())
gg_tag.loc[gg_tag['gg_50plus'].notnull()]

,parcel_id,geometry,index_right,Shape_Leng,Shape_Area,gg_50plus
0,1906302.0,POINT (515952.701 4243055.845),NaN,NaN,NaN,NaN
1,1740249.0,POINT (605099.029 4222734.303),NaN,NaN,NaN,NaN
2,1630642.0,POINT (617528.748 4133218.640),NaN,NaN,NaN,NaN
3,1725445.0,POINT (630054.383 4108236.029),NaN,NaN,NaN,NaN
4,837988.0,POINT (536304.276 4201510.130),NaN,NaN,NaN,NaN


,parcel_id,geometry,index_right,Shape_Leng,Shape_Area,gg_50plus
81,1296016.0,POINT (588416.918 4140330.159),0.0,31.073629,0.104814,GG
122,344076.0,POINT (590220.784 4150685.530),0.0,31.073629,0.104814,GG
160,1841543.0,POINT (591916.696 4240719.616),0.0,31.073629,0.104814,GG
209,1855643.0,POINT (589601.401 4232562.030),0.0,31.073629,0.104814,GG
271,37.0,POINT (559967.540 4185433.464),0.0,31.073629,0.104814,GG
...,...,...,...,...,...,...
1956194,1299900.0,POINT (574739.586 4144982.499),0.0,31.073629,0.104814,GG
1956198,1386886.0,POINT (600106.034 4134554.919),0.0,31.073629,0.104814,GG
1956199,141351.0,POINT (564963.039 4180832.469),0.0,31.073629,0.104814,GG
1956204,269036.0,POINT (579926.210 4168835.977),0.0,31.073629,0.104814,GG


In [55]:
gg_tag.loc[gg_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,Shape_Leng,Shape_Area,gg_50plus


In [56]:
# PPA
ppa_layer = gpd.read_file(dataset_path, driver='fileGDB', layer='PPA_ExportFeatures')
# ppa_layer.rename(columns={'PBA50_Geo': 'tra_50plus'}, inplace=True)

print(ppa_layer.crs)
ppa_layer_proj = ppa_layer.to_crs('EPSG:26910')
print(ppa_layer_proj.crs) 

epsg:4326
EPSG:26910


In [57]:
ppa_layer_proj.head()

,globalid,county,jurisdicti,ppa_name,acres,Shape_Length,Shape_Area,geometry
0,5c999288-43b9-4341-8486-0fda50aefbe7,Alameda,Hayward,Hayward PPA,3853.585071,0.409479,0.001593,"MULTIPOLYGON (((583016.305 4162295.064, 583015..."
1,71078b70-384d-4d7c-b36c-6aaffc7d8a11,Alameda,Fremont,Pacific Commons Priority Production Area,1135.219690,0.156762,0.000468,"MULTIPOLYGON (((591407.098 4151453.065, 591554..."
2,664f58d6-50b8-413d-9d5d-dd5b982987de,Alameda,Fremont,Bayside Industrial Priority Production Area,1542.309641,0.166956,0.000636,"MULTIPOLYGON (((592847.758 4149986.328, 592848..."
3,b356bd35-9cdf-47ae-b037-39ef000da0f9,Alameda,Livermore,Westside PPA,300.899323,0.045386,0.000124,"MULTIPOLYGON (((604523.055 4170446.761, 604514..."
4,43226b47-01eb-4430-888a-ee40bdbbd9e6,Contra Costa,Concord,Western Concord PPA,112.784479,0.028528,0.000040,"MULTIPOLYGON (((585474.708 4201101.567, 585477..."


In [58]:
ppa_layer_proj['ppa_50plus'] = ppa_layer_proj[['jurisdicti', 'ppa_name']].apply(' - '.join, axis=1)

In [59]:
# tag parcels by PPA
ppa_tag = gpd.sjoin(p10_centroid_gdf, ppa_layer_proj[['geometry', 'ppa_50plus']], how="left", predicate="within")
display(ppa_tag.head())
ppa_tag.loc[ppa_tag['ppa_50plus'].notnull()]

,parcel_id,geometry,index_right,ppa_50plus
0,1906302.0,POINT (515952.701 4243055.845),NaN,NaN
1,1740249.0,POINT (605099.029 4222734.303),NaN,NaN
2,1630642.0,POINT (617528.748 4133218.640),NaN,NaN
3,1725445.0,POINT (630054.383 4108236.029),NaN,NaN
4,837988.0,POINT (536304.276 4201510.130),NaN,NaN


,parcel_id,geometry,index_right,ppa_50plus
122,344076.0,POINT (590220.784 4150685.530),1.0,Fremont - Pacific Commons Priority Production ...
209,1855643.0,POINT (589601.401 4232562.030),23.0,Suisun City - East Side PPA
271,37.0,POINT (559967.540 4185433.464),7.0,Oakland - Port PPA
314,606851.0,POINT (582726.023 4204894.110),12.0,Unincorporated Contra Costa - Pacheco Manufact...
446,1732230.0,POINT (581987.068 4232745.284),31.0,Fairfield - Fairfield PPA
...,...,...,...,...
1953869,161267.0,POINT (573183.072 4173530.648),8.0,Oakland - Airport PPA
1954607,998638.0,POINT (552530.285 4177467.247),19.0,San Francisco - Bayshore/Central Waterfront/Is...
1955368,322363.0,POINT (594602.134 4147763.932),2.0,Fremont - Bayside Industrial Priority Producti...
1955412,172469.0,POINT (571809.707 4174994.419),8.0,Oakland - Airport PPA


In [60]:
ppa_tag.loc[ppa_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,ppa_50plus


In [61]:
## PDA layer
pda_layer = gpd.read_file(dataset_path, driver='fileGDB', layer='PBA50Plus_PDA_ExportFeatures_correct')
# pda_layer.rename(columns={'PBA50_Geo': 'gg_50plus'}, inplace=True)

print(pda_layer.crs)
pda_layer_proj = pda_layer.to_crs('EPSG:26910')
print(pda_layer_proj.crs)

epsg:26910
epsg:26910


In [62]:
pda_layer_proj.head()

,join_key,county,jurisdicti,pda_name,Shape_Length,Shape_Area,geometry
0,230ec884-82c6-4074-a3e2-877bb1cea97b,Alameda,Berkeley,North Berkeley BART,924.160358,5.083622e+04,MULTIPOLYGON Z (((562890.077 4192145.677 0.000...
1,0a13cdcd-d2e8-46e1-8652-60d54f28c843,Alameda,Fremont,Irvington Transit PDA,9112.144618,3.246482e+06,MULTIPOLYGON Z (((591571.522 4155139.993 0.000...
2,a75db8bd-4a27-4288-b878-24cee994e148,Alameda,Fremont,Centerville Transit PDA,6233.925919,2.354421e+06,MULTIPOLYGON Z (((588181.418 4157988.943 0.000...
3,84b5bd67-82eb-4202-9a66-a978d2218197,Alameda,Fremont,Downtown/City CenterTransit PDA,8862.169884,3.719576e+06,MULTIPOLYGON Z (((590239.195 4156026.173 0.000...
4,f50e7d2c-0fce-4fb6-83bb-7c037aa9446a,Alameda,Fremont,Warm Springs Innovation District Transit PDA,9082.987773,3.691505e+06,MULTIPOLYGON Z (((593518.505 4152009.076 0.000...


In [63]:

pda_layer_proj['pda_50plus'] = pda_layer_proj[['jurisdicti', 'pda_name']].apply(' - '.join, axis=1)

# tag parcels by PDA
pda_tag = gpd.sjoin(p10_centroid_gdf, pda_layer_proj[['geometry', 'pda_50plus']], how="left", predicate="within")
display(pda_tag.head())
pda_tag.loc[pda_tag['pda_50plus'].notnull()]

,parcel_id,geometry,index_right,pda_50plus
0,1906302.0,POINT (515952.701 4243055.845),NaN,NaN
1,1740249.0,POINT (605099.029 4222734.303),NaN,NaN
2,1630642.0,POINT (617528.748 4133218.640),NaN,NaN
3,1725445.0,POINT (630054.383 4108236.029),NaN,NaN
4,837988.0,POINT (536304.276 4201510.130),NaN,NaN


,parcel_id,geometry,index_right,pda_50plus
160,1841543.0,POINT (591916.696 4240719.616),54.0,Fairfield - Fairfield-Vacaville Train Station
286,636171.0,POINT (586893.241 4206891.459),171.0,Concord - Community Reuse Area/Los Medanos
383,224680.0,POINT (607743.932 4170994.699),97.0,Livermore - Downtown
384,2000853.0,POINT (530037.460 4235785.873),169.0,Petaluma - Corona
390,1288099.0,POINT (593829.288 4139479.929),137.0,San Jose - North San Jose
...,...,...,...,...
1956190,1674412.0,POINT (605943.303 4122230.645),217.0,San Jose - Santa Clara Valley Transportation A...
1956192,2050178.0,POINT (593096.607 4130816.819),198.0,San Jose - Valley Fair Santana Row
1956198,1386886.0,POINT (600106.034 4134554.919),218.0,San Jose - East Santa Clara/Alum Rock Corridor
1956204,269036.0,POINT (579926.210 4168835.977),91.0,Hayward - The Cannery


In [64]:
pda_tag.loc[pda_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,pda_50plus
501675,760456.0,POINT (542838.893 4201542.363),196.0,San Rafael - South East San Rafael Canal PDA
1633394,1551872.0,POINT (599664.184 4130759.512),217.0,San Jose - Santa Clara Valley Transportation A...


In [65]:
pda_tag.drop_duplicates(subset=['parcel_id'], inplace=True)

In [66]:
pda_tag.loc[pda_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,pda_50plus


In [67]:
# HRA data
hra_geo = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\HRA_final_2023_shapefile\final_2023_public.shp')
hra_geo

,fips,fips_bg,cnty_nm,countyd,region,ecn_dmn,env_hl_,ed_domn,index,oppcat,geometry
0,06001400100,None,Alameda,06001,Bay Area Region,0.796927,0.927856,0.787575,0.590095,Highest Resource,"POLYGON ((-122.23418 37.85254, -122.23428 37.8..."
1,06001400200,None,Alameda,06001,Bay Area Region,0.960588,0.627255,0.810955,0.701320,Highest Resource,"POLYGON ((-122.25253 37.85110, -122.25094 37.8..."
2,06001400300,None,Alameda,06001,Bay Area Region,0.927188,0.619238,0.324649,0.210942,High Resource,"POLYGON ((-122.26417 37.83787, -122.26427 37.8..."
3,06001400400,None,Alameda,06001,Bay Area Region,0.872411,0.657983,0.523046,0.358539,High Resource,"POLYGON ((-122.26081 37.84869, -122.26023 37.8..."
4,06001400500,None,Alameda,06001,Bay Area Region,0.759519,0.698063,0.566466,0.310218,High Resource,"POLYGON ((-122.26023 37.85274, -122.26081 37.8..."
...,...,...,...,...,...,...,...,...,...,...,...
10131,06115041000,061150410005,Yuba,06115,Rural Areas,NaN,NaN,NaN,NaN,None,"POLYGON ((-121.63631 39.24941, -121.63581 39.2..."
10132,06115041100,061150411001,Yuba,06115,Rural Areas,0.292683,0.878049,0.414634,-0.079483,Moderate Resource,"POLYGON ((-121.27892 39.49773, -121.27724 39.5..."
10133,06115041100,061150411002,Yuba,06115,Rural Areas,0.536585,0.878049,0.829268,0.321475,Highest Resource,"POLYGON ((-121.32330 39.36354, -121.32129 39.3..."
10134,06115041100,061150411003,Yuba,06115,Rural Areas,0.073171,0.878049,0.658537,-0.165002,Moderate Resource,"POLYGON ((-121.28387 39.46216, -121.27864 39.4..."


In [68]:
# clean up the data a little bit
print(hra_geo['oppcat'].unique())
hra_dict = {'Highest Resource': 'HRA',
            'High Resource': 'HRA'}
hra_geo['hra_50plus'] = hra_geo['oppcat'].map(hra_dict)
hra_geo = hra_geo.loc[hra_geo['hra_50plus'] == 'HRA']
hra_geo

['Highest Resource' 'High Resource' 'Moderate Resource' 'Low Resource'
 'High Segregation & Poverty' None]


,fips,fips_bg,cnty_nm,countyd,region,ecn_dmn,env_hl_,ed_domn,index,oppcat,geometry,hra_50plus
0,06001400100,None,Alameda,06001,Bay Area Region,0.796927,0.927856,0.787575,0.590095,Highest Resource,"POLYGON ((-122.23418 37.85254, -122.23428 37.8...",HRA
1,06001400200,None,Alameda,06001,Bay Area Region,0.960588,0.627255,0.810955,0.701320,Highest Resource,"POLYGON ((-122.25253 37.85110, -122.25094 37.8...",HRA
2,06001400300,None,Alameda,06001,Bay Area Region,0.927188,0.619238,0.324649,0.210942,High Resource,"POLYGON ((-122.26417 37.83787, -122.26427 37.8...",HRA
3,06001400400,None,Alameda,06001,Bay Area Region,0.872411,0.657983,0.523046,0.358539,High Resource,"POLYGON ((-122.26081 37.84869, -122.26023 37.8...",HRA
4,06001400500,None,Alameda,06001,Bay Area Region,0.759519,0.698063,0.566466,0.310218,High Resource,"POLYGON ((-122.26023 37.85274, -122.26081 37.8...",HRA
...,...,...,...,...,...,...,...,...,...,...,...,...
10127,06115041000,061150410001,Yuba,06115,Rural Areas,1.000000,0.121951,0.512195,0.231388,Highest Resource,"POLYGON ((-121.41390 39.22871, -121.40965 39.2...",HRA
10128,06115041000,061150410002,Yuba,06115,Rural Areas,0.487805,0.121951,0.804878,0.102659,High Resource,"POLYGON ((-121.46947 39.33348, -121.46865 39.3...",HRA
10129,06115041000,061150410003,Yuba,06115,Rural Areas,0.951220,0.121951,0.878049,0.542419,Highest Resource,"POLYGON ((-121.51084 39.23150, -121.51027 39.2...",HRA
10130,06115041000,061150410004,Yuba,06115,Rural Areas,0.878049,0.121951,0.487805,0.095763,High Resource,"POLYGON ((-121.60128 39.21917, -121.57788 39.2...",HRA


In [69]:
print(hra_geo.crs)
hra_geo_proj = hra_geo.to_crs('EPSG:26910')
print(hra_geo_proj.crs) 

epsg:4326
EPSG:26910


In [70]:
# tag parcels by HRA
hra_tag = gpd.sjoin(p10_centroid_gdf, hra_geo_proj[['geometry', 'hra_50plus']], how="left", predicate="within")
display(hra_tag.head())
hra_tag.loc[hra_tag['hra_50plus'].notnull()]

,parcel_id,geometry,index_right,hra_50plus
0,1906302.0,POINT (515952.701 4243055.845),9263.0,HRA
1,1740249.0,POINT (605099.029 4222734.303),NaN,NaN
2,1630642.0,POINT (617528.748 4133218.640),NaN,NaN
3,1725445.0,POINT (630054.383 4108236.029),NaN,NaN
4,837988.0,POINT (536304.276 4201510.130),4311.0,HRA


,parcel_id,geometry,index_right,hra_50plus
0,1906302.0,POINT (515952.701 4243055.845),9263.0,HRA
4,837988.0,POINT (536304.276 4201510.130),4311.0,HRA
11,1596090.0,POINT (595573.476 4112310.238),8703.0,HRA
12,537189.0,POINT (577650.313 4202636.182),698.0,HRA
13,550054.0,POINT (606656.037 4182681.621),782.0,HRA
...,...,...,...,...
1956194,1299900.0,POINT (574739.586 4144982.499),8691.0,HRA
1956196,839102.0,POINT (543691.510 4191754.180),4335.0,HRA
1956197,839104.0,POINT (543696.543 4191760.428),4335.0,HRA
1956199,141351.0,POINT (564963.039 4180832.469),161.0,HRA


In [71]:
hra_tag.loc[hra_tag['parcel_id'].duplicated()]

,parcel_id,geometry,index_right,hra_50plus
717057,1593346.0,POINT (596144.121 4118553.626),8703.0,HRA
759294,1611984.0,POINT (602629.058 4113427.675),8706.0,HRA
778720,793512.0,POINT (537934.777 4201076.801),4338.0,HRA
1015033,837962.0,POINT (534851.088 4203749.171),4338.0,HRA
1024482,742497.0,POINT (534672.171 4204601.029),4338.0,HRA
1475344,793482.0,POINT (538048.749 4201040.048),4338.0,HRA
1579359,793481.0,POINT (538012.464 4201065.600),4338.0,HRA
1681043,742609.0,POINT (534846.407 4204430.418),4338.0,HRA


In [72]:
hra_tag.drop_duplicates(subset=['parcel_id'], inplace=True)

In [73]:
# UGB layer
ugb_tag = pd.read_csv(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\urbansim_2022_p10-urbansim_ugb-20231208.csv')
print(ugb_tag.shape[0])
print(ugb_tag['parcel_id'].nunique())
ugb_tag.head()

1956208
1956208


,Unnamed: 0,parcel_id,ugb_class
0,0,717795.0,UGB
1,1,155346.0,CL
2,2,2052462.0,USL
3,3,1374667.0,USL
4,4,1585530.0,NaN


In [74]:
# EPC layer
epc_layer = gpd.read_file(r'C:\Users\ywang\Downloads\DRAFT_Equity_Priority_Communities_-_Plan_Bay_Area_2050_Plus\DRAFT_Equity_Priority_Communities_Plan_Bay_Area_2050__polygons.shp')
epc_layer.head()

,tract_geoi,county_fip,tot_pop_mi,tot_pop_se,tot_pop_po,tot_pop_ci,tot_hh,pop_zvhhs,tot_fam,tot_pop_ov,...,minori_1_2,spfam_1_2,disab_1_2,lep_1_2,below2_1_2,zvhh_1_2,hus_re_1_2,epc_2050p,epc_class,geometry
0,06075017602,75,3218,3218,3218,3218,1643,1206,442,3218,...,1,0,1,0,0,1,1,0,NA,"POLYGON ((-13626723.174 4548282.654, -13626475..."
1,06013307202,13,4531,4531,4497,4531,1570,192,1193,4133,...,1,1,1,0,1,0,1,1,Highest,"POLYGON ((-13562749.644 4580113.521, -13562647..."
2,06075017603,75,4509,4509,4452,4509,2280,1456,764,4356,...,1,0,0,1,0,1,1,0,NA,"POLYGON ((-13627466.566 4547914.777, -13627319..."
3,06013308002,13,4287,4287,4272,4282,1416,72,1134,4126,...,0,0,1,0,0,0,0,0,NA,"POLYGON ((-13557192.686 4579164.756, -13555289..."
4,06075017700,75,1989,1989,1975,1989,783,188,351,1917,...,0,1,0,0,0,1,0,0,NA,"POLYGON ((-13627587.793 4547780.277, -13627409..."


In [75]:
epc_layer = epc_layer[['epc_2050p', 'geometry']]
print(epc_layer.crs)
epc_layer_proj = epc_layer.to_crs('EPSG:26910')
print(epc_layer_proj.crs) 

epsg:3857
EPSG:26910


In [76]:
# tag parcels by EPC
epc_tag = gpd.sjoin(p10_centroid_gdf, epc_layer_proj, how="left", predicate="within")
display(epc_tag.head())
display(epc_tag.loc[epc_tag['epc_2050p'] == 1])
display(epc_tag.loc[hra_tag['parcel_id'].duplicated()])
epc_tag.drop_duplicates(subset=['parcel_id'], inplace=True)

epc_tag['epc_50plus'] = 'NA'
epc_tag.loc[epc_tag['epc_2050p'] == 1, 'epc_50plus'] = 'EPC'

,parcel_id,geometry,index_right,epc_2050p
0,1906302.0,POINT (515952.701 4243055.845),1052.0,0.0
1,1740249.0,POINT (605099.029 4222734.303),581.0,0.0
2,1630642.0,POINT (617528.748 4133218.640),1617.0,0.0
3,1725445.0,POINT (630054.383 4108236.029),1617.0,0.0
4,837988.0,POINT (536304.276 4201510.130),1307.0,0.0


,parcel_id,geometry,index_right,epc_2050p
62,479378.0,POINT (603699.489 4200931.587),968.0,1.0
134,653525.0,POINT (593849.344 4208695.812),545.0,1.0
259,630095.0,POINT (617273.477 4207021.933),1687.0,1.0
291,1975709.0,POINT (501202.372 4257235.685),1074.0,1.0
340,532720.0,POINT (616695.027 4203544.754),1687.0,1.0
...,...,...,...,...
1956156,1548693.0,POINT (598265.138 4132792.091),349.0,1.0
1956164,468179.0,POINT (557063.530 4199614.181),1130.0,1.0
1956166,1564264.0,POINT (602409.458 4135696.286),701.0,1.0
1956182,264889.0,POINT (580570.873 4170306.131),1242.0,1.0


,parcel_id,geometry,index_right,epc_2050p


In [77]:
# write out for QAQC on a map

# add these columns back to parcel
p10_pba50plus_tag = p10_gdf.loc[:, p10_gdf.columns != 'centroid']
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(tra_tag[['parcel_id', 'tra_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(gg_tag[['parcel_id', 'gg_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(pda_tag[['parcel_id', 'pda_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(ppa_tag[['parcel_id', 'ppa_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(hra_tag[['parcel_id', 'hra_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(ugb_tag[['parcel_id', 'ugb_class']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

p10_pba50plus_tag = p10_pba50plus_tag.merge(epc_tag[['parcel_id', 'epc_50plus']], on='parcel_id', how='left')
print(p10_pba50plus_tag.shape[0])

1956207
1956207
1956207
1956207
1956207
1956207
1956207
1956207


In [78]:
print(type(p10_pba50plus_tag))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [79]:
p10_pba50plus_tag.to_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\p10_tagging_20231212.shp')

C:\Users\ywang\AppData\Local\Temp\ipykernel_24496\1405906641.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  p10_pba50plus_tag.to_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\p10_tagging_20231212.shp')


In [80]:
print(list(p10_pba50plus_tag))

['parcel_id', 'county', 'jurisdiction', 'acres', 'x', 'y', 'is_multipolygon', 'Shape_Length', 'Shape_Area', 'geometry', 'tra_50plus', 'gg_50plus', 'pda_50plus', 'ppa_50plus', 'hra_50plus', 'ugb_class', 'epc_50plus']


In [81]:
## "exd_id" field

p10_df = pd.read_csv(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\p10_raw_data_from_PBA50\p10.csv')
b10_df = pd.read_csv(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\p10_raw_data_from_PBA50\b10.csv')


C:\Users\ywang\AppData\Local\Temp\ipykernel_24496\719652361.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  p10_df = pd.read_csv(r'C:\Users\ywang\Documents\ArcGIS\Projects\pba50plus_GrowthGeographies_p10tagging\p10_raw_data_from_PBA50\p10.csv')


In [82]:
p10_df.loc[p10_df['PARCEL_ID'] == 2054505]

,OBJECTID,PARCEL_ID,DEVELOPMEN,LAND_VALUE,ACRES,COUNTY_ID,ZONE_ID,PROPORTION,TAX_EXEMPT,APN,...,zoningmodcat,perffoot,perfarea,mapshell,tpa_id,perfarea2,alt_zoning,zonetype,Shape_Length,Shape_Area


In [83]:
print(list(p10_df))
print(list(b10_df))

['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_county', 'jurisdiction', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodcat', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'Shape_Length', 'Shape_Area']
['OBJECTID', 'building_id', 'parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'id']


In [84]:
p10_df['tpp_id'].unique()

array([nan, 'b1', 'b2', 'lrt1', 'b3', 'bart2', 'bart3', 'brt1', 'bart1',
       'brt2', 'brt3', 'lrt2', 'lrt3', 'cr3', 'cr2', 'cr4'], dtype=object)

In [85]:
b10 = b10_df[['parcel_id', 'residential_units']]

In [86]:
b10.dtypes

parcel_id            int64
residential_units    int64
dtype: object

In [87]:
print(b10.shape[0])
print(b10['parcel_id'].nunique())

1843351
1843292


In [88]:
b10_agg = b10.groupby('parcel_id')[['residential_units']].sum().reset_index()
b10_agg.loc[b10_agg['residential_units'] > 0, 'exd_id'] = 'exd'
b10_agg['PARCEL_ID'] = b10_agg['parcel_id']
b10_agg

,parcel_id,residential_units,exd_id,PARCEL_ID
0,26,0,nan,26
1,27,0,nan,27
2,28,0,nan,28
3,29,0,nan,29
4,30,0,nan,30
...,...,...,...,...
1843287,2054496,0,nan,2054496
1843288,2054501,0,nan,2054501
1843289,2054503,0,nan,2054503
1843290,2054504,0,nan,2054504


## Integrate with other fields from PBA50

In [89]:
# other fields from PBA50
parcels_geograpy_pba50 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\PBA50 Final Blueprint Large General Input Data\2021_01_12_parcels_geography.csv')

C:\Users\ywang\AppData\Local\Temp\ipykernel_24496\73347543.py:2: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  parcels_geograpy_pba50 = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\PBA50 Final Blueprint Large General Input Data\2021_01_12_parcels_geography.csv')


In [90]:
print(list(parcels_geograpy_pba50))

['PARCEL_ID', 'geom_id', 'jurisdiction_id', 'juris_name_full', 'juris_id', 'juris', 'ACRES', 'pda_id_pba40', 'tpp_id', 'exp_id', 'opp_id', 'zoningmodcat', 'perffoot', 'perfarea', 'urbanized', 'hra_id', 'trich_id', 'cat_id', 'zoninghzcat', 'gg_id', 'pda_id_pba50_db', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'exsfd_id', 'pba50zoningmodcat', 'pba50chcat', 'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_sesit_id', 'fbp_ppa_id', 'fbp_exp2020_id', 'fbp_exsfd_id', 'fbpzoningmodcat', 'fbpchcat', 'nodev']


In [91]:
sesit_dis_dict = {
    'DIS': 'DIS',
    'HRADIS': 'DIS'
}
parcels_geograpy_pba50['dis_id'] = parcels_geograpy_pba50['fbp_sesit_id'].map(sesit_dis_dict)

In [92]:
parcels_geograpy_pba50plus = parcels_geograpy_pba50[['PARCEL_ID', 'geom_id', 'jurisdiction_id','juris', 'ACRES', 
'tpp_id', 'opp_id', 'perffoot', 'perfarea', 'urbanized', 'fbp_exp2020_id', 'fbp_exsfd_id',
'dis_id', 'nodev',
# 'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 'fbp_ppa_id'
]]

parcels_geograpy_pba50plus.rename(columns = {
    'fbp_exp2020_id': 'exp2020_id',
    'fbp_exsfd_id': 'exsfd_id'}, inplace = True)

C:\Users\ywang\AppData\Local\Temp\ipykernel_24496\443844864.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcels_geograpy_pba50plus.rename(columns = {


In [93]:
# add Growth Geographies fields

p10_pba50plus_tag['ugb_id'] = 'NA'
p10_pba50plus_tag.loc[p10_pba50plus_tag['ugb_class'].notnull(), 'ugb_id'] = 'UGB'

p10_pba50plus_tag['ppa_id'] = 'NA'
p10_pba50plus_tag.loc[p10_pba50plus_tag['ppa_50plus'].notnull(), 'ppa_id'] = 'PPA'


geography_update = p10_pba50plus_tag[['parcel_id', 'tra_50plus', 'gg_50plus', 'pda_50plus', 'hra_50plus', 'ppa_id', 'ugb_id', 'epc_50plus']]
geography_update.rename(columns = {
    'parcel_id': 'PARCEL_ID',
    'tra_50plus': 'tra_id', 
    'gg_50plus': 'gg_id', 
    'pda_50plus': 'pda_id', 
    'hra_50plus': 'hra_id',
    'epc_50plus': 'epc_id'}, inplace=True)

parcels_geograpy_pba50plus = parcels_geograpy_pba50plus.merge(geography_update, on='PARCEL_ID', how='left')

C:\Users\ywang\AppData\Local\Temp\ipykernel_24496\1997498799.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geography_update.rename(columns = {


In [94]:
# add residentil unit column
parcels_geograpy_pba50plus = parcels_geograpy_pba50plus.merge(b10_agg[['exd_id', 'PARCEL_ID']], on = 'PARCEL_ID', how='left')


In [95]:
parcels_geograpy_pba50plus

,PARCEL_ID,geom_id,jurisdiction_id,juris,ACRES,tpp_id,opp_id,perffoot,perfarea,urbanized,...,dis_id,nodev,tra_id,gg_id,pda_id,hra_id,ppa_id,ugb_id,epc_id,exd_id
0,229116,10305106092872,41992,livermore,3.360520,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,HRA,NA,UGB,NA,NaN
1,244166,11107351665227,41992,livermore,1.294423,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,NaN,NA,UGB,NA,NaN
2,202378,11030175960628,33000,hayward,14.993605,NaN,NaN,1,0,0,...,NaN,0.0,NaN,NaN,NaN,NaN,NA,UGB,NA,exd
3,2004420,6381677629073,97,unincorporated_sonoma,316.247146,NaN,NaN,0,0,0,...,DIS,0.0,NaN,NaN,NaN,NaN,NA,NA,NA,nan
4,340332,314875459798,26000,fremont,0.621275,b1,NaN,1,1,1,...,DIS,1.0,NaN,NaN,NaN,HRA,NA,UGB,NA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,17158666132196,2252,antioch,0.071424,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,NaN,NA,UGB,EPC,exd
1956204,594355,16389503450045,16000,concord,0.137534,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,NaN,NA,UGB,NA,exd
1956205,804156,1496694834659,52582,novato,0.019658,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,NaN,NA,UGB,NA,NaN
1956206,646401,10694584892329,13,unincorporated_contra_costa,0.254764,NaN,NaN,1,0,1,...,DIS,0.0,NaN,NaN,NaN,HRA,NA,UGB,NA,exd


In [96]:
print(list(parcels_geograpy_pba50plus))

['PARCEL_ID', 'geom_id', 'jurisdiction_id', 'juris', 'ACRES', 'tpp_id', 'opp_id', 'perffoot', 'perfarea', 'urbanized', 'exp2020_id', 'exsfd_id', 'dis_id', 'nodev', 'tra_id', 'gg_id', 'pda_id', 'hra_id', 'ppa_id', 'ugb_id', 'epc_id', 'exd_id']


In [97]:
for i in ['exp2020_id', 'exsfd_id', 'dis_id', 'tra_id', 'gg_id', 'pda_id', 'hra_id', 'ppa_id', 'ugb_id', 'exd_id', 'epc_id']:
    parcels_geograpy_pba50plus[i].fillna('NA', inplace=True)

display(parcels_geograpy_pba50plus)

,PARCEL_ID,geom_id,jurisdiction_id,juris,ACRES,tpp_id,opp_id,perffoot,perfarea,urbanized,...,dis_id,nodev,tra_id,gg_id,pda_id,hra_id,ppa_id,ugb_id,epc_id,exd_id
0,229116,10305106092872,41992,livermore,3.360520,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,HRA,NA,UGB,NA,NA
1,244166,11107351665227,41992,livermore,1.294423,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,NA,NA,UGB,NA,NA
2,202378,11030175960628,33000,hayward,14.993605,NaN,NaN,1,0,0,...,NA,0.0,NA,NA,NA,NA,NA,UGB,NA,exd
3,2004420,6381677629073,97,unincorporated_sonoma,316.247146,NaN,NaN,0,0,0,...,DIS,0.0,NA,NA,NA,NA,NA,NA,NA,nan
4,340332,314875459798,26000,fremont,0.621275,b1,NaN,1,1,1,...,DIS,1.0,NA,NA,NA,HRA,NA,UGB,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,17158666132196,2252,antioch,0.071424,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,NA,NA,UGB,EPC,exd
1956204,594355,16389503450045,16000,concord,0.137534,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,NA,NA,UGB,NA,exd
1956205,804156,1496694834659,52582,novato,0.019658,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,NA,NA,UGB,NA,NA
1956206,646401,10694584892329,13,unincorporated_contra_costa,0.254764,NaN,NaN,1,0,1,...,DIS,0.0,NA,NA,NA,HRA,NA,UGB,NA,exd


In [98]:
parcels_geograpy_pba50plus.to_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\p10 Datasets for PBA2050plus\parcels_geography_2023_12_12.csv', index=False)